# Lab03: Linear systems of equations

First some helper code for generating good matrices for LU factorisation tests. 

In [ ]:
import numpy as np
from scipy.sparse import diags


def generate_safe_system(n, seed=None):
    """
    Generate a linear system A x = b where A is strictly diagonally dominant,
    ensuring LU factorization without pivoting will work.

    Parameters:
        n (int): Size of the system (n x n)

    Returns:
        A (ndarray): n x n strictly diagonally dominant matrix
        b (ndarray): RHS vector
        x_true (ndarray): The true solution vector
    """

    k = [np.ones(n - 1), -2 * np.ones(n), np.ones(n - 1)]
    offset = [-1, 0, 1]
    A = diags(k, offset).toarray()

    # Solution is always all ones
    x_true = np.ones((n, 1))

    # Compute b = A @ x_true
    b = A @ x_true

    return A, b, x_true

## Exercise 1: LU factorisation

1. Implement the algorithm for LU factorisation as described [in the
notes](https://comp2870-2526.github.io/linear-algebra-notes/src/lec04.html#sec-LU-factorisation).
You should implement a Python function which accepts an $n \times n$
matrix $A$ represented as a numpy array and returns one lower triangular
matrix $L$ and one upper triangular matrix $U$ with $A = LU$.

    Here is some starter code:

In [ ]:
def lu_factorisation(A):
    """
    Compute the LU factorisation of a square matrix A.

    The function decomposes a square matrix ``A`` into the product of a lower
    triangular matrix ``L`` and an upper triangular matrix ``U`` such that:

    .. math::
        A = L U

    where ``L`` has unit diagonal elements and ``U`` is upper triangular.

    Parameters
    ----------
    A : numpy.ndarray
        A 2D NumPy array of shape ``(n, n)`` representing the square matrix to
        factorise.

    Returns
    -------
    L : numpy.ndarray
        A lower triangular matrix with shape ``(n, n)`` and unit diagonal.
    U : numpy.ndarray
        An upper triangular matrix with shape ``(n, n)``.
    """
    n, m = A.shape
    if n != m:
        raise ValueError(f"Matrix A is not square {A.shape=}")

    # construct arrays of zeros
    L, U = np.zeros_like(A), np.zeros_like(A)

    # ...

2. Test your solution on the matrix $A$: $$
A = \begin{pmatrix}
4 & 2 & 0 \\ 2 & 3 & 1 \\ 0 & 1 & 2.5
\end{pmatrix}.
$$ First compute the LU factorisation by hand and then
check $A = L U$.

3. Consider the large matrix given by:

In [ ]:
A_large, b_large, x_large = generate_safe_system(100)

We can use the LU factorisation as another way to compute the determinant of a matrix.
We can use the fact that we can easily compute the determinant of triangular matrix and that $\det(XY) = \det(X) \det(Y)$.

Use this following code to determine the determinant of `A`:

In [ ]:
def determinant(A):
    n = A.shape[0]
    L, U = lu_factorisation(A)

    det_L = 1.0
    det_U = 1.0

    for i in range(n):
        det_L *= L[i, i]
        det_U *= U[i, i]

    return det_L * det_U

## Exercise 2: Timing LU factorisation

Combine your implementation of LU factorisation with the forwards and
backwards solve from the lecture notes to create a solver for a system
of linear equations. Compare how long it takes for each part of your
code to run against the code for Gaussian elimination from the lecture
notes. Describe what you find.

Use the following code to help generate the test cases:

In [ ]:
sizes = [2**j for j in range(1, 6)]

for n in sizes:
    # generate a random system of linear equations of size n
    A, b, x = generate_safe_system(n)

    # do the solve
    ...

<div class="alert alert-block alert-danger">
<b>Portfolio exercise</b>
You should submit:

1. Your code for `lu_factorisation`
2. The determinant of `A_large` from Exercise 1.3.
3. The plot of run times for LU factorisation (your code) and Gaussian elimination (from the notes)
4. A comment on the differences between the run times.
</div>


## Exercise 3: LU factorisation with pivoting (Extension)

We can improve the stability of LU factorisation by additionally
including a pivoting step.

The new algorithm looks like:

TODO! update algorithm steps here

1.  Initialise $P = I_n$ (the $n \times n$ identity), $L = O_n$ (the
    $n \times n$ zero matrix) and $U = O_n$.

2.  For $k = 1, \ldots n$:

    1.  Find the pivot row with the largest $|A_{ik}|$ for
        $i = k, \ldots, n$. Swap rows $k$ and $p$ in $A$ and store this
        swap in $P$.
    2.  Continue with LU factorisation in column $k$ as usual.
    3.  Repeat for next column $k+1$.

Implement this new method and test on the system of linear equations
given by: $$
\begin{pmatrix}
10 & -7 & 0 \\ -3 & 2.1 - \varepsilon & 6 \\ 5 & -1 & 5
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\x_3
\end{pmatrix}
= \begin{pmatrix}
7 \\ 9.9 + \varepsilon \\ 11
\end{pmatrix}.
$$ Test with $\varepsilon = 10^{-14}$.